# Database APIs for ASER

Firstly, install the necessay packages

In [ ]:
# !pip install -r requirements.txt 

Then, install the aser package.

In [ ]:
# !python setup.py develop

In [1]:
# Loading
import numpy as np
import random
import json

import keras.preprocessing.text as keras_text
from keras import optimizers
from keras.preprocessing.text import Tokenizer
import sklearn.preprocessing as sklp
from keras import backend as K

import random, os, sys
import numpy as np
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.initializers import *
import tensorflow as tf
from keras.engine.topology import Layer


from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.optimizers import Adam

from keras import initializers, regularizers, constraints, optimizers, layers


Using TensorFlow backend.


Now, we can import the necessay APIs to access the database

In [2]:
# from aser.database.db_API import KG_Connection
# from aser.database.db_API import generate_event_id, generate_relation_id, generate_id

We provide three different modes: *insert*, *cache*, and *memory*.
* *insert* mode supports the insert operation.
* *cache* mode load necessary metadata in the memory
* *memory* mode loads the whole data in the memory

However, it still takes a lot of time, and costs a lot memory due to the large size.
We use the *cache* mode to build a DB connection.

In [3]:
# kg_conn = KG_Connection(db_path=r'./KG_v0.1.0.db', mode='cache')

In [4]:
# print('SIZE:', 'eventualities: ', len(kg_conn.event_id_set), 'relations:', len(kg_conn.relation_id_set))

Noted that eventualites extracted from differnent sentences are identified as the unique one in the database if their words are exactly the same, and relations between two eventualities are identified as the unique one distribution in the database. 

We can print the attributes of eventualites and relations.

In [5]:
# print(list(zip(kg_conn.event_columns, kg_conn.event_column_types)))

In [6]:
# print(list(zip(kg_conn.relation_columns, kg_conn.relation_column_types)))

We can retrieve eventualities from the database. 
The extractor is not released at present, we just use simple words to check.

In [7]:
# kg_conn.get_exact_match_event(generate_id('i learn python'))

Relations can also be retrieved.

In [8]:
# kg_conn.get_exact_match_relation([generate_id('i be tired'), generate_id('i sleep')])

In [9]:
# print(kg_conn.get_exact_match_event("3df702fe2ef4b4084d63eac65fa3254f102172d3"))

In [10]:
#####################################
## Step 1: Create a sample of KG   ##
#####################################

# # notice: it is a very sparse graph



# # Sampling relations and corresponding events
# sample_size = 1000
# relation_id_list = list(kg_conn.relation_id_set)

# sample_inds = list(range(len(relation_id_list)))
# random.shuffle(sample_inds)

# sample_ids = []
# for ind in sample_inds[0:sample_size]:
#     sample_ids.append(relation_id_list[ind])

# sample_relations = dict()
# sample_events = dict()



# for sample_id in sample_ids:
    
#     relation = kg_conn.get_exact_match_relation(sample_id)
    
#     event1_id = relation["event1_id"]
#     event2_id = relation["event2_id"]
    
#     event1 = kg_conn.get_exact_match_event(event1_id)
#     event2 = kg_conn.get_exact_match_event(event2_id)
    
#     key  = event1["words"] + " && " + event2["words"]
    
#     sample_relations[key] = relation
#     sample_events[event1_id] = event1
#     sample_events[event2_id] = event2


# print("# of relations: ", str(len(sample_relations.keys())))
# print("# of event: ", str(len(sample_events.keys())))
    
# sample = dict()
# sample["relation"] = sample_relations
# sample["events"] = sample_events
# # Dump to json for later use

# with open('samples.json', 'w') as fp:
#     json.dump(sample, fp)

In [11]:
with open('samples.json') as fp:
    samples = json.load(fp)

In [12]:
events = samples["events"]
relations = samples["relation"]

# treat events as nodes first
event_list = list(events.values())

event_id_list = []

for event in event_list:
    event_id_list.append(event["_id"])
    
# prepare the edge list
edge_list = []

for relation in relations.values():
    edge = []
    event1_id = relation["event1_id"]
    event2_id = relation["event2_id"]
    weight = relation['Co_Occurrence']
    
    edge.append(event_id_list.index(event1_id))
    edge.append(event_id_list.index(event2_id))
    edge.append(weight)
    
    edge_list.append(edge)
    

In [13]:
###########################################################
##           Step 2: Adapted Skipgram + Deepwalk         ##
###########################################################

import numpy as np
import networkx as nx
import random


class Graph():
    def __init__(self, nx_G, is_directed, p, q):
        self.G = nx_G
        self.is_directed = is_directed
        self.p = p
        self.q = q

    def node2vec_walk(self, walk_length, start_node):
        '''
        Simulate a random walk starting from start node.
        '''
        G = self.G
        alias_nodes = self.alias_nodes
        alias_edges = self.alias_edges

        walk = [start_node]

        while len(walk) < walk_length:
            cur = walk[-1]
            cur_nbrs = sorted(G.neighbors(cur))
            if len(cur_nbrs) > 0:
                if len(walk) == 1:
                    walk.append(cur_nbrs[alias_draw(alias_nodes[cur][0], alias_nodes[cur][1])])
                else:
                    prev = walk[-2]
                    next = cur_nbrs[alias_draw(alias_edges[(prev, cur)][0],
                                               alias_edges[(prev, cur)][1])]
                    walk.append(next)
            else:
                break

        return walk

    def simulate_walks(self, num_walks, walk_length):
        '''
        Repeatedly simulate random walks from each node.
        '''
        G = self.G
        walks = []
        nodes = list(G.nodes())
        print('Walk iteration:')
        for walk_iter in range(num_walks):
            print(str(walk_iter + 1), '/', str(num_walks))
            random.shuffle(nodes)
            for node in nodes:
                walks.append(self.node2vec_walk(walk_length=walk_length, start_node=node))

        return walks

    def get_alias_edge(self, src, dst):
        '''
        Get the alias edge setup lists for a given edge.
        '''
        G = self.G
        p = self.p
        q = self.q

        unnormalized_probs = []
        for dst_nbr in sorted(G.neighbors(dst)):
            if dst_nbr == src:
                unnormalized_probs.append(G[dst][dst_nbr]['weight'] / p)
            elif G.has_edge(dst_nbr, src):
                unnormalized_probs.append(G[dst][dst_nbr]['weight'])
            else:
                unnormalized_probs.append(G[dst][dst_nbr]['weight'] / q)
        norm_const = sum(unnormalized_probs)
        normalized_probs = [float(u_prob) / norm_const for u_prob in unnormalized_probs]

        return alias_setup(normalized_probs)

    def preprocess_transition_probs(self):
        '''
        Preprocessing of transition probabilities for guiding the random walks.
        '''
        G = self.G
        is_directed = self.is_directed

        alias_nodes = {}
        for node in G.nodes():
            unnormalized_probs = [G[node][nbr]['weight'] for nbr in sorted(G.neighbors(node))]
            norm_const = sum(unnormalized_probs)
            normalized_probs = [float(u_prob) / norm_const for u_prob in unnormalized_probs]
            alias_nodes[node] = alias_setup(normalized_probs)

        alias_edges = {}
        triads = {}

        if is_directed:
            for edge in G.edges():
                alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
        else:
            for edge in G.edges():
                alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
                alias_edges[(edge[1], edge[0])] = self.get_alias_edge(edge[1], edge[0])

        self.alias_nodes = alias_nodes
        self.alias_edges = alias_edges

        return


def alias_setup(probs):
    '''
    Compute utility lists for non-uniform sampling from discrete distributions.
    Refer to https://hips.seas.harvard.edu/blog/2013/03/03/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
    for details
    '''
    K = len(probs)
    q = np.zeros(K)
    J = np.zeros(K, dtype=np.int)

    smaller = []
    larger = []
    for kk, prob in enumerate(probs):
        q[kk] = K * prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)

    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()

        J[small] = large
        q[large] = q[large] + q[small] - 1.0
        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)

    return J, q


def alias_draw(J, q):
    '''
    Draw sample from a non-uniform discrete distribution using alias sampling.
    '''
    K = len(J)

    kk = int(np.floor(np.random.rand() * K))
    if np.random.rand() < q[kk]:
        return kk
    else:
        return J[kk]


In [14]:
# Prepare the walks

def get_G_from_edges(edges):
    edge_dict = dict()
    # calculate the count for all the edges
    for edge in edges:
        edge_key = str(edge[0]) + '_' + str(edge[1])
        if edge_key not in edge_dict:
            edge_dict[edge_key] = edge[2]
        else:
            edge_dict[edge_key] += edge[2]
    tmp_G = nx.DiGraph()
    
    for edge_key in edge_dict:
        weight = edge_dict[edge_key]
        # add edges to the graph
        tmp_G.add_edge(edge_key.split('_')[0], edge_key.split('_')[1])
        # add weights for all the edges
        tmp_G[edge_key.split('_')[0]][edge_key.split('_')[1]]['weight'] = weight
    return tmp_G

directed = True
p = 1
q = 1
num_walks = 10
walk_length = 10
dimension = 200
iterations = 50

train_edges = edge_list

G = Graph(get_G_from_edges(train_edges), directed, p, q)
G.preprocess_transition_probs()
walks = G.simulate_walks(num_walks, walk_length)

Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


In [15]:
# test skipgram

# couples, labels = sequence.skipgrams(np.arange(3), vocabulary_size=3)
# for couple in couples:
#     assert couple[0] in [0, 1, 2] and couple[1] in [0, 1, 2]

# # test window size and categorical labels
# data = np.array([np.arange(5), np.arange(5)])

# print(numerical_walks[0])
# couples, labels = sequence.skipgrams(numerical_walks[0], vocabulary_size=2, window_size=1,
#                             categorical=True)
# print(couples)
print()

In [16]:
# Prepare Training Data
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.preprocessing.sequence as sequence


def event2word(event_id_list,event_list_original, tokenizer):
    event_word = []
    for index in event_id_list:
        event = event_list_original[index]
        event_word.append(event["words"])
        
    event_word = tokenizer.texts_to_sequences(event_word)
    event_word = pad_sequences(event_word)
    
    return event_word


max_features = 1790 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 10 # max number of words of a event
embed_size = 300 # how big is each word vector
event_size = 1000 # number of eventualities invovled
vocab_size = max_features
features_dim = embed_size 


# event list -  edge_lsit
window_size = 5
event_size = len(event_list)

# collect all the words in the events
event_words_collection = []
for event in event_list:
    event_words_collection.append(event["words"])
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(event_words_collection)

numerical_walks = []
for walk in walks:
    if len(walk) > 1:
        numerical_walks.append(list(map(int, walk)))

# sampling_table = sequence.make_sampling_table(event_size)
couples = []
labels = []

# temp_walk = numerical_walks
# np.shuffle(temp_walk)
# sample_walk = temp_walk[0:2000]


for walk in numerical_walks:
    temp_couples, temp_labels = sequence.skipgrams(walk, event_size, window_size=5)
    couples.extend(temp_couples)
    labels.extend(temp_labels)
    
labels = np.array(labels, dtype="int32")

event_target, event_context = zip(*couples)
target_input = event2word(event_target, event_list, tokenizer)
context_input = event2word(event_context, event_list, tokenizer)
        

In [17]:
# preparing embedding matrix

EMBEDDING_FILE = 'glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding='utf-8'))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= (max_features): continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

C:\Users\xzhao\Anaconda3\envs\Keras\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  import sys


In [31]:
class SkipgramModel:
    def __init__(self, max_features, embed_size, embedding_matrix):
        vocab_size = max_features
        features_dim = embed_size
        
        target_input = Input(shape=(None,), name='target_input')
        context_input = Input(shape=(None,), name='context_input')
        
        #embedding_layer = Embedding(vocab_size, features_dim, input_length=1, name='embedding_layer')
        
        embedding_layer = Embedding(vocab_size, features_dim, weights=[embedding_matrix])
        
        #encode target words
        target_encoded = embedding_layer(target_input)
        # target_encoded = Reshape((features_dim, 1))(target_encoded)
        target_encoded = Bidirectional(CuDNNLSTM(128, return_sequences=False))(target_encoded)

        #encode context words   
        context_encoded = embedding_layer(context_input)
        # context_encoded = Reshape((features_dim, 1))(context_encoded)
        context_encoded = Bidirectional(CuDNNLSTM(128, return_sequences=False))(context_encoded)
        
        #dot product two seqs
        
        dot_product = dot([target_encoded, context_encoded], axes=1)
        
        # dot_product = Reshape((1,))(dot_product)

        #normalize
        output_layer = Dense(1, activation='sigmoid', name='output_layer')(dot_product)
        
        self.main_model = Model(inputs=[target_input, context_input], outputs=[output_layer])

        similarity = dot([target_encoded, context_encoded], axes=0, normalize=True)
        self.embedding_model = Model(inputs=[target_input, context_input], outputs=[target_encoded])
        self.similarity_model = Model(inputs=[target_input, context_input], outputs=[similarity])

In [44]:
skipgram = SkipgramModel(vocab_size, features_dim, embedding_matrix)
skipgram.main_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
skipgram.main_model.fit([target_input, context_input], labels,epochs=2, verbose=2)
skipgram.main_model.save_weights('skipgram_weights.h5')

Epoch 1/2
 - 22s - loss: 0.3479 - acc: 0.8246
Epoch 2/2
 - 20s - loss: 0.0706 - acc: 0.9767


In [45]:
# Getting the emebdding vectors
skipgram = SkipgramModel(vocab_size, features_dim, embedding_matrix)
skipgram.main_model.load_weights('skipgram_weights.h5')

out = skipgram.embedding_model.predict_on_batch([target_input[:2],context_input[:2]])

# event_list
# event_id_list
# edge_list

embedding_dic = dict()

temp = np.array(range(len(event_list)))
out = skipgram.embedding_model.predict_on_batch([temp,temp])

for i in range(len(out)):
    embedding_dic[event_id_list[i]] = list(out[i])


In [65]:
###########################################################
##                     Step 3: Evalution                 ##
###########################################################

# Link Prediction
def randomly_choose_false_edges(nodes, true_edges):
    tmp_list = list()
    all_edges = list()
    for i in range(len(nodes)):
        for j in range(len(nodes)):
            all_edges.append((i, j))
    random.shuffle(all_edges)
    for edge in all_edges:
        if edge[0] == edge[1]:
            continue
        if (nodes[edge[0]], nodes[edge[1]]) not in true_edges and (nodes[edge[1]], nodes[edge[0]]) not in true_edges:
            tmp_list.append((nodes[edge[0]], nodes[edge[1]]))
    return tmp_list


# Compare with original score
from sklearn.metrics import roc_auc_score

def get_AUC(embedding_dic, true_edges, false_edges, event_id_list):
    true_list = list()
    prediction_list = list()
    for edge in true_edges:
        tmp_score = get_neighbourhood_score(embedding_dic, event_id_list[edge[0]], event_id_list[edge[1]])
        true_list.append(1)
        prediction_list.append(tmp_score)

    for edge in false_edges:
        tmp_score = get_neighbourhood_score(embedding_dic, event_id_list[edge[0]], event_id_list[edge[1]])
        true_list.append(0)
        prediction_list.append(tmp_score)
        
    y_true = np.array(true_list)
    y_scores = np.array(prediction_list)
    return roc_auc_score(y_true, y_scores)

def get_neighbourhood_score(embedding_dic, node1_id, node2_id):
    try:
        vector1 = embedding_dic[node1_id]
        vector2 = embedding_dic[node2_id]
        return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    except:
        return random.random()
    
# event_list
# edge_list
mask_event_list = range(len(event_list))
mask_edge_list = []

for i in range(len(edge_list)):
    mask_edge_list = edge_list[:2]

negative_edges = randomly_choose_false_edges(mask_event_list, mask_edge_list)[:1000]
tmp_AUC_score = get_AUC(embedding_dic, edge_list, negative_edges,event_id_list)
print('tmp_accuracy:', tmp_AUC_score)

tmp_accuracy: 0.6323895


In [64]:
a = [1,2,3]
b = a[:2]
print(b)

[1, 2]
